In [ ]:
!pip install SpeechRecognition
!pip install os-sys
!pip install deepgram-sdk==0.3.0
!pip install pydub

#!pip uninstall whisper
#!pip uninstall openai-whisper
!pip install openai-whisper


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of os-sys to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of os-sys to determine which version is compatible with other requirements. This could take a while.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━

**To Improve Transcription Let's use: Deepgram-sdk, Wisper and pydub**

In [ ]:
#5b8b243e83906934f93c55d141d9d95315239b12

In [ ]:
import os
import requests
import whisper
from pydub import AudioSegment

def preprocess_audio(wav_path):
    # Load the wav file
    audio = AudioSegment.from_wav(wav_path)

    # Normalize the audio (adjust volume)
    normalized_audio = audio.normalize()

    # Save the processed file to a new path
    processed_wav_path = wav_path.replace(".wav", "_processed.wav")
    normalized_audio.export(processed_wav_path, format="wav")

    return processed_wav_path

def wav_to_text_whisper(wav_dir, txt_dir):
    # Create the output directory if it doesn't exist
    if not os.path.exists(txt_dir):
        os.mkdir(txt_dir)

    # Load the Whisper model
    model = whisper.load_model("base")

    # Iterate over files in the wav directory
    for file in os.listdir(wav_dir):
        # Process only .wav files
        if file.endswith(".wav"):
            wav_path = os.path.join(wav_dir, file)

            # Preprocess the audio (normalize)
            processed_wav_path = preprocess_audio(wav_path)

            # Transcribe using Whisper
            result = model.transcribe(processed_wav_path)
            text = result['text']

            # Save transcription
            txt_path = os.path.join(txt_dir, file[:-4] + ".txt")
            with open(txt_path, "w") as text_file:
                text_file.write(text)
            print(f"Transcribed {file} successfully using Whisper.")

def wav_to_text_deepgram(wav_dir, txt_dir, deepgram_api_key):
    # Create the output directory if it doesn't exist
    if not os.path.exists(txt_dir):
        os.mkdir(txt_dir)

    # Iterate over files in the wav directory
    for file in os.listdir(wav_dir):
        # Process only .wav files
        if file.endswith(".wav"):
            wav_path = os.path.join(wav_dir, file)

            # Open the audio file for Deepgram transcription
            with open(wav_path, 'rb') as f:
                response = requests.post(
                    f"https://api.deepgram.com/v1/listen?access_token={deepgram_api_key}",
                    headers={"Authorization": f"Bearer {deepgram_api_key}"},
                    files={"file": f}
                )
                if response.status_code == 200:
                    text = response.json()['results']['channels'][0]['alternatives'][0]['transcript']
                    txt_path = os.path.join(txt_dir, file[:-4] + ".txt")
                    with open(txt_path, "w") as text_file:
                        text_file.write(text)
                    print(f"Transcribed {file} successfully using Deepgram.")
                else:
                    print(f"Error transcribing {file} with Deepgram: {response.status_code}")

if __name__ == "__main__":
    # Directory where your .wav files are stored
    wav_dir = r"/content/drive/MyDrive/Dutch wav"

    # Directory where you want to save the transcribed text files
    txt_dir = r"/content/drive/MyDrive/Dutch Transcription"

    # For Deepgram, include your API key
    deepgram_api_key = "5b8b243e83906934f93c55d141d9d95315239b12"

    # Choose whether to use Whisper or Deepgram
    wav_to_text_whisper(wav_dir, txt_dir)  # Uncomment this line to use Whisper
    # wav_to_text_deepgram(wav_dir, txt_dir, deepgram_api_key)  # Uncomment this line to use Deepgram


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 125MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcribed Dutch_Hockey.wav successfully using Whisper.
